<a href="https://colab.research.google.com/github/mayasrikanth/CS-236G-Project/blob/main/Generate_Story_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Working on code for in-context prompt generation (currently debugging!)

In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 12.1 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 6.8 MB 40.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import transformers

In [3]:
# Using HuggingFace GPT-Neo (open-source alternative to GPT-3, large GPT-J) to generate story prompts 
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")  

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [7]:
prompt = (
    "Sentence: A black cat sits next to a pumpkin" 
    "Image: A cat with fluffy black fur sits on a stone wall in the backyard next to a carved Halloween pumpkin "
    "Sentence: A swan on a lake "
)

In [8]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

In [9]:
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
gen_text = tokenizer.batch_decode(gen_tokens)[0]

In [13]:
gen_text = tokenizer.batch_decode(gen_tokens)

In [14]:
print(gen_text)

['Sentence: A black cat sits next to a pumpkinImage: A cat with fluffy black fur sits on a stone wall in the backyard next to a carved Halloween pumpkin Sentence: A swan on a lake   Image 1of 1 - The black cat sitting on the pumpkinImage 2of 1Image 3 of 1\n\n\n\nA black cat sits next to a pumpkinImage: A cat with fluffy black fur sits on a stone wall in the backyard next to a carved Halloween pumpkin\n\nA']
